In [1]:
import pandas as pd

In [2]:
newMapping = pd.read_csv('./character_data.csv')

In [3]:
smallData = newMapping[['ID','SFBASE__STATUS__C','SFBASE__SKU__C','L1_BUSINESS_LINE__C','L2_PRODUCT_LINE__C',
            'L3_PRODUCT_FAMILY__C','L4_PRODUCT_OFFERING__C','NAME','ISACTIVE','Quotable','SFDCAPPROVALMATRIXPRODUCT__C']]

In [4]:
newCols = list(smallData.columns)
newCols[3] = 'Level1'
newCols[4] = 'Level2'
newCols[5] = 'Level3'
newCols[6] = 'Level4'

In [5]:
smallData.columns = newCols

In [6]:
#smallData = newMapping[newMapping.columns[0:15]]
smallData.shape

(39, 11)

In [7]:
class myNode(object):
    def __inint__(self,initdata,mylevel):
        self.val = initdata
        self.level = mylevel
        self.SKU = None
        self.id = None
        self.status = None
        self.matrix = None
        self.children = {}

In [8]:
activeProds = smallData[smallData['SFBASE__STATUS__C']!='Obsolete']

In [9]:
activeProds['SFDCAPPROVALMATRIXPRODUCT__C'] = activeProds['SFDCAPPROVALMATRIXPRODUCT__C'].apply(lambda x: str(x).replace(" ","%20"))

In [10]:
activeProds.shape

(39, 11)

In [11]:
blankValue = "BLANK"
activeProds.loc[activeProds['Level1']=='0','Level1'] = blankValue
activeProds.loc[activeProds['Level2']=='0','Level2'] = blankValue
activeProds.loc[activeProds['Level3']=='0','Level3'] = blankValue
activeProds.loc[activeProds['Level4']=='0','Level4'] = blankValue

activeProds.loc[activeProds['Level1'].isnull(),'Level1'] = blankValue
activeProds.loc[activeProds['Level2'].isnull(),'Level2'] = blankValue
activeProds.loc[activeProds['Level3'].isnull(),'Level3'] = blankValue
activeProds.loc[activeProds['Level4'].isnull(),'Level4'] = blankValue

In [12]:
activeProds['Has a Blank']=(activeProds['Level1']==blankValue) | (activeProds['Level2']==blankValue) | (activeProds['Level3']==blankValue) | (activeProds['Level4']==blankValue)

In [13]:
cleanData = activeProds[activeProds['Has a Blank']==False]

In [14]:
cleanData = cleanData[cleanData['SFBASE__STATUS__C']!="Testing"].reset_index(drop=True)

In [15]:
cleanData=cleanData.reset_index(drop=True)

In [16]:
cleanData.shape

(39, 12)

In [17]:
cleanData.head()

,ID,SFBASE__STATUS__C,SFBASE__SKU__C,Level1,Level2,Level3,Level4,NAME,ISACTIVE,Quotable,SFDCAPPROVALMATRIXPRODUCT__C,Has a Blank
0,X1,Kit Harington,Male,Protagonist,High Born,Stark,Warrior,Jon Snow,X,X,Jon_Snow,False
1,X2,Sophie Turner,Female,Protagonist,High Born,Stark,Politician,Sansa Stark,X,X,Sansa_Stark,False
2,X3,Emilia Clarke,Female,Protagonist,High Born,Targaryen,Politician,Daenerys Targaryen,X,X,Daenerys_Targaryen,False
3,X4,Maisie Williams,Female,Protagonist,High Born,Stark,Warrior,Arya Stark,X,X,Arya_Stark,False
4,X5,Peter Dinklage,Male,Protagonist,High Born,Lannister,Politician,Tyrion Lannister,X,X,Tyrion_Lannister,False


In [18]:
list(cleanData.columns)

['ID',
 'SFBASE__STATUS__C',
 'SFBASE__SKU__C',
 'Level1',
 'Level2',
 'Level3',
 'Level4',
 'NAME',
 'ISACTIVE',
 'Quotable',
 'SFDCAPPROVALMATRIXPRODUCT__C',
 'Has a Blank']

In [19]:
cleanData.loc[[10]]

,ID,SFBASE__STATUS__C,SFBASE__SKU__C,Level1,Level2,Level3,Level4,NAME,ISACTIVE,Quotable,SFDCAPPROVALMATRIXPRODUCT__C,Has a Blank
10,X11,Rory McCann,Male,Protagonist,High Born,Clegane,Warrior,Sandor Clegane,X,X,Sandor_Clegane,False


In [20]:
def buildTree(root,cleanData):
    for row in range(cleanData.shape[0]):
        l1 = l2 = l3 = l4 = None
        if cleanData.loc[row]['Level1'] in root.children:
            l1 = root.children[cleanData.loc[row]['Level1']]
        else:
            l1 = myNode()
            l1.val = cleanData.loc[row]['Level1']
            l1.level = 0
            l1.children = {}
            l1.SKU = None
            l1.id = None
            l1.status = None
            l1.matrix = None
            root.children[cleanData.loc[row]['Level1']] = l1
        
        if cleanData.loc[row]['Level2'] in l1.children:
            l2 = l1.children[cleanData.loc[row]['Level2']]
        else:
            l2 = myNode()
            l2.val = cleanData.loc[row]['Level2']
            l2.level = 0
            l2.children = {}
            l2.SKU = None
            l2.id = None
            l2.status = None
            l2.matrix = None
            l1.children[cleanData.loc[row]['Level2']] = l2
        
        if cleanData.loc[row]['Level3'] in l2.children:
            l3 = l2.children[cleanData.loc[row]['Level3']]
        else:
            l3 = myNode()
            l3.val = cleanData.loc[row]['Level3']
            l3.level = 0
            l3.children = {}
            l3.SKU = None
            l3.id = None
            l3.status = None
            l3.matrix = None
            l2.children[cleanData.loc[row]['Level3']] = l3
        
        if cleanData.loc[row]['Level4'] in l3.children:
            l4 = l3.children[cleanData.loc[row]['Level4']]
        else:
            l4 = myNode()
            l4.val = cleanData.loc[row]['Level4']
            l4.level = 0
            l4.children = {}
            l4.SKU = None
            l4.id = None
            l4.status = None
            l4.matrix = None
            l3.children[cleanData.loc[row]['Level4']] = l4
    
        childNode = myNode()
        childNode.val = cleanData.loc[row]['NAME']
        childNode.level = 5
        childNode.children = {}
        childNode.SKU = cleanData.loc[row]['SFBASE__SKU__C']
        childNode.id = cleanData.loc[row]['ID']
        childNode.status = cleanData.loc[row]['SFBASE__STATUS__C']
        childNode.matrix = cleanData.loc[row]['SFDCAPPROVALMATRIXPRODUCT__C']
        l4.children[cleanData.loc[row]['NAME']] = childNode

In [21]:
root = myNode()
root.val = "Root"
root.level = 0
root.SKU = None
root.children = {}
root.id = None
root.status = None

buildTree(root,cleanData)

In [22]:
#phaseOut = cleanData[cleanData.SFBASE__STATUS__C.str.contains("Phase")].reset_index(drop=True)
active = cleanData[cleanData['SFBASE__SKU__C']=="Male"].reset_index(drop=True)
quotable = cleanData[cleanData['SFBASE__SKU__C']=="Female"].reset_index(drop=True)
#restricted = cleanData[cleanData['SFBASE__STATUS__C']=="Restricted"].reset_index(drop=True)
#testing = cleanData[cleanData['SFBASE__STATUS__C']=="Testing"].reset_index(drop=True)

In [23]:
phaseOut[phaseOut['SFBASE__SKU__C']==210000908]

NameError: name 'phaseOut' is not defined

In [24]:
cleanData[cleanData['SFBASE__SKU__C']==210000908]

,ID,SFBASE__STATUS__C,SFBASE__SKU__C,Level1,Level2,Level3,Level4,NAME,ISACTIVE,Quotable,SFDCAPPROVALMATRIXPRODUCT__C,Has a Blank


In [25]:
allTree = myNode()
allTree.val = "Characters"
allTree.level = 0
allTree.SKU = None
allTree.children = {}
allTree.id = None
allTree.status = None

phaseOutTree = myNode()
phaseOutTree.val = "Characters"
phaseOutTree.level = 0
phaseOutTree.SKU = None
phaseOutTree.children = {}
phaseOutTree.id = None
phaseOutTree.status = None

activeTree = myNode()
activeTree.val = "Characters"
activeTree.level = 0
activeTree.SKU = None
activeTree.children = {}
activeTree.id = None
activeTree.status = None

quotableTree = myNode()
quotableTree.val = "Characters"
quotableTree.level = 0
quotableTree.SKU = None
quotableTree.children = {}
quotableTree.id = None
quotableTree.status = None

"""
restrictedTree = myNode()
restrictedTree.val = "Catalog"
restrictedTree.level = 0
restrictedTree.SKU = None
restrictedTree.children = {}
restrictedTree.id = None
restrictedTree.status = None
*/
testingTree = myNode()
testingTree.val = "Catalog"
testingTree.level = 0
testingTree.SKU = None
testingTree.children = {}
testingTree.id = None
testingTree.status = None/*"""

'\nrestrictedTree = myNode()\nrestrictedTree.val = "Catalog"\nrestrictedTree.level = 0\nrestrictedTree.SKU = None\nrestrictedTree.children = {}\nrestrictedTree.id = None\nrestrictedTree.status = None\n*/\ntestingTree = myNode()\ntestingTree.val = "Catalog"\ntestingTree.level = 0\ntestingTree.SKU = None\ntestingTree.children = {}\ntestingTree.id = None\ntestingTree.status = None/*'

In [26]:
buildTree(allTree,cleanData)
#buildTree(phaseOutTree,phaseOut)
buildTree(activeTree,active)
#buildTree(restrictedTree,restricted)
#buildTree(testingTree,testing)
buildTree(quotableTree,quotable)

In [27]:
phaseOutTree.children['Marketing Cloud'].children['Social Studio'].children['Social Core Editions'].children['SELA'].children

KeyError: 'Marketing Cloud'

In [28]:
cleanData.shape

(39, 12)

In [29]:
phaseOut.shape

NameError: name 'phaseOut' is not defined

In [30]:
active.shape

(25, 12)

In [31]:

quotable.shape

(14, 12)

In [32]:
def printTreeHelp(varName,rootNode,baseString,indent):
    return printTree(rootNode,"var "+varName+" =\n",indent,[0])
def printTree(rootNode,baseString,indent,indexList):
    indent1 = indent + "  "
    baseString += indent + "{\n"
    baseString += indent1 + '"name": ' + '"' + rootNode.val + '"'
    if rootNode.SKU != None: #and False:  #uncomment the False to remove SKU
        baseString += ",\n" + indent1 + '"sku": ' + '"' + str(rootNode.SKU) + '",\n'
        baseString += indent1 + '"id": "' + str(rootNode.id) + '",\n'
        baseString += indent1 + '"matrix": "' +rootNode.matrix + '",\n'
        baseString += indent1 + '"status": "' + rootNode.status + '"\n'
    else:
        baseString += ",\n"
        baseString += indent1 + '"id": "' +str(indexList[0]) + '"\n'
        indexList[0] += 1
    if rootNode.children != {}:
        baseString = baseString[:-1] + ",\n"
        baseString += indent1 + '"children": [\n'
        for key in sorted(rootNode.children.keys()):
            baseString += printTree(rootNode.children[key],"",indent+"    ",indexList)
            baseString += ",\n"
        baseString = baseString[:-2] + "\n"
        baseString += indent1 + "]\n"
    baseString += indent + "}"
    return baseString

In [33]:
a=printTreeHelp("all",allTree,"","")
#b=printTreeHelp("phase",phaseOutTree,"","")
c=printTreeHelp("male",activeTree,"","")
#d=printTreeHelp("testing",testingTree,"","")
#e=printTreeHelp("restricted",restrictedTree,"","")
f=printTreeHelp("female",quotableTree,"","")

In [34]:
print a
#print b
print c
#print d
#print e
print f

var all =
{
  "name": "Characters",
  "id": "0",
  "children": [
    {
      "name": "Antagonist",
      "id": "1",
      "children": [
        {
          "name": "Commoner",
          "id": "2",
          "children": [
            {
              "name": "Other",
              "id": "3",
              "children": [
                {
                  "name": "Politician",
                  "id": "4",
                  "children": [
                    {
                      "name": "Petyr Baelish",
                      "sku": "Male",
                      "id": "X36",
                      "matrix": "Petyr_Baelish",
                      "status": "Aidan Gillen"
                    }
                  ]
                }
              ]
            }
          ]
        },
        {
          "name": "High Born",
          "id": "5",
          "children": [
            {
              "name": "Bolton",
              "id": "6",
              "children": [
                {
         

In [76]:
phaseOut.shape

(1774, 11)

In [58]:

buildTree(phaseOutTree,phaseOut)
b=printTreeHelp("phase",phaseOutTree,"","")

In [61]:
b=printTreeHelp("phase",phaseOutTree,"","")
print b

var phase =
{
  "name": "Product Catalog",
  "children": [
    {
      "name": "Allocation Cloud",
      "children": [
        {
          "name": "Adjustment",
          "children": [
            {
              "name": "Adjustment",
              "children": [
                {
                  "name": "Adjustment",
                  "children": [
                    {
                      "name": "Adjustment - Buddy Media",
                      "sku": "100004549",
                      "id": "01t30000001GswIAAS",
                      "status": "Phase Out"
                    },
                    {
                      "name": "Adjustment - BuyBuddy",
                      "sku": "100004551",
                      "id": "01t30000001GswSAAS",
                      "status": "Phase Out"
                    },
                    {
                      "name": "Adjustment - Radian6",
                      "sku": "100004219",
                      "id": "01t30000001GmbqAAC",
    

# Other Stuff

In [15]:
outputData = {}

In [16]:
editData = cleanData.copy()
editData["Count of SKUs"] = editData["ID"]
tableCounts = pd.DataFrame(editData.groupby(['Level1','Level2','Level3','Level4'])['Count of SKUs'].count())
tableCounts = tableCounts.reset_index().reset_index().sort('Count of SKUs',ascending=False)
updateColumns = list(tableCounts.columns)
updateColumns[0] = "Index"
tableCounts.columns = updateColumns
outputData['SKU Counts'] = tableCounts.copy()

C:\Users\kdwan\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:4: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [ ]:
def writeToExcel(filename,myDict,myOrder,showIndex=False):
    writer = pd.ExcelWriter(filename+'.xlsx')
    for tab in myOrder:
        myDict[tab].to_excel(writer,sheet_name=tab,index=showIndex)
    writer.save()

In [ ]:
writeToExcel('Summary Data in Each Category',outputData,sorted(outputData.keys()))